In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from torch.utils.data import TensorDataset, DataLoader
import copy
# from class_models import early_stop

In [2]:
# setting device and reproducibility
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(42)
print(f"Using device: {device}")

Using device: cpu


### Learning schedule to find the best learning rate using *ReduceOnPlateau* scheduler
The learning rate scheduler (ReduceLROnPlateau) adjusts the learning rate based on validation loss. The use of ReduceLROnPlateau scheduler helps in adjusting the learning rate dynamically based on the validation loss, which can improve the training process. This training task tracks the best model in the k-fold. The best model is tracked by comparing the validation loss of each epoch. The learning rate (best_lr) that resulted in the lowest validation loss is recorded.

In [3]:
# import data 
data_frame = pd.read_csv("data/NYCTaxiFares.csv", na_values=["NA", "?"])

In [4]:
# function to calculate the distance of the travel
def haversine_distance(dat_f, lat1, lon1, lat2, lon2):
    
    # average radius of the Earth in (km)
    r = 6371
    
    phi1 = np.radians(dat_f[lat1])
    phi2 = np.radians(dat_f[lat2])
    delta_phi = np.radians(dat_f[lat2] - dat_f[lat1])
    delta_lambda = np.radians(dat_f[lon2] - dat_f[lon1])
    
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = (r * c)
    
    return d

def preprocessing(df_n, cat_cols):
    """
    Preprocesses the data and adds pandas categorical fields to a dataframe.
    :param df_n: pandas dataframe 
    :param cat_cols: list of categorical fields
    :return: pandas dataframe
    """
    # append a 'dist_km' new feature in the dataframe
    df_n['dist_km'] = haversine_distance(df_n, 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude')
    
    # remove outliers
    dfd = df_n[(df_n['fare_amount'] != 49.57) & (df_n['fare_amount'] != 45.00)].copy()
    
    # convert to pd datetime
    dfd['pickup_datetime'] = pd.to_datetime(dfd['pickup_datetime'])
    
    # Correcting pickup_datetime due to daylight savings time (April)
    dfd['EDTdate'] = dfd['pickup_datetime'] - pd.Timedelta(hours=4)
    
    # create new time fields
    dfd['Hour'] = dfd['EDTdate'].dt.hour
    dfd['AMorPM'] = np.where(dfd['Hour']<12, 'am', 'pm')
    dfd['Weekday'] = dfd['EDTdate'].dt.strftime("%a")
    
    # transform to pandas categorical variables
    for cat in cat_cols:
        dfd[cat] = dfd[cat].astype('category')
    
    dfd = dfd.drop(columns=['pickup_datetime'])
    
    return dfd

def model_tensors(df, cat_cols, cont_cols, y_col):
    """
    Get categorical, continuous and label tensors for the model
    :param df: pd dataframe
    :param cat_cols: list of categorical fields
    :param cont_cols: list of continuous fields
    :param y_col: list with the labels
    :return: cats, conts, y tensors
    """
    
    # group the data in categorical continuous and target label    
    cats = np.stack([df[col].cat.codes.values for col in cat_cols], axis=1)
    conts = np.stack([df[col].values for col in cont_cols], axis=1)
    y = df[y_col].values.reshape(-1, 1)

    # Convert to PyTorch tensors
    cats_t = torch.tensor(cats, dtype=torch.int64).to(device)
    conts_t = torch.tensor(conts, dtype=torch.float32).to(device)
    y_t = torch.tensor(y, dtype=torch.float32).to(device)
    
    return cats_t, conts_t, y_t

def create_embedding_sizes(df, cat_cols):
    """
    Create embedding sizes for PyTorch embedding layers
    :param df: pandas dataframe
    :param cat_cols: list of categorical fields
    :return: emb_sizes list
    """
    # categorical sizes list
    cat_sizes = [len(df[col].cat.categories) for col in cat_cols]

    # embedding sizes list (divide the number of unique entries in each column by two, if the result is greater than 50 select 50)
    emb_sizes = [(size, min(50,(size+1)//2)) for size in cat_sizes]
    
    return emb_sizes



In [5]:
df = preprocessing(data_frame, ['Hour', 'AMorPM', 'Weekday'])

cats, conts, y = model_tensors(df, ['Hour', 'AMorPM', 'Weekday'], ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'dist_km'], ['fare_amount'])

# number of continuous fields of the conts tensor
n_cont = conts.shape[1]

emb_sizes = create_embedding_sizes(df, ['Hour', 'AMorPM', 'Weekday'])

### Model definition

In [6]:
# Define the model
class TabularModel(nn.Module):
    def __init__(self, emb_sizes, n_cont, out_size, layers, p=0.5):
        super().__init__()
        self.embeds = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in emb_sizes])
        self.emb_drop = nn.Dropout(p)
        self.batch_norm_cont = nn.BatchNorm1d(n_cont)

        layer_list = []
        n_emb = sum([nf for ni, nf in emb_sizes])
        n_in = n_emb + n_cont
        for i in layers:
            layer_list.append(nn.Linear(n_in, i))
            layer_list.append(nn.ReLU(inplace=True))
            layer_list.append(nn.BatchNorm1d(i))
            layer_list.append(nn.Dropout(p))
            n_in = i

        layer_list.append(nn.Linear(layers[-1], out_size))
        self.layers = nn.Sequential(*layer_list)

    def forward(self, x_cat, x_cont):
        embeddings = [e(x_cat[:, i]) for i, e in enumerate(self.embeds)]
        x = torch.cat(embeddings, 1)
        x = self.emb_drop(x)
        x_cont = self.batch_norm_cont(x_cont)
        x = torch.cat([x, x_cont], 1)
        x = self.layers(x)
        return x

### K-folds function generator
Since the *train_index* and the *val_index* are those who effectively creates the training subsets we can create all the six subsets: 2 for categorical, 2 for continuous and 2 for y labels.

In [7]:
# Function-generator to split the data into k folds
def kfold_split(k, X_cat, X_cont, y):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    for train_index, val_index in kf.split(X_cat):
        X_cat_train, X_cat_val = X_cat[train_index], X_cat[val_index]
        X_cont_train, X_cont_val = X_cont[train_index], X_cont[val_index]
        y_train, y_val = y[train_index], y[val_index]
        yield X_cat_train, X_cat_val, X_cont_train, X_cont_val, y_train, y_val

### Model and training parameters and hyperparameters 

In [8]:
# emb_sizes
# cont
out_size = 1 # <- for regression task
layers = [200, 100]
p = 0.2 # dropout probability
batch_size = 1024
epochs = 100

### Cross-Validation loop

In [9]:
# Cross-validation loop
k = 12
fold = 0
best_lr = None
best_loss = float('inf')

for X_cat_train, X_cat_val, X_cont_train, X_cont_val, y_train, y_val in kfold_split(k, cats, conts, y):
    fold += 1
    print(f"Fold {fold}/{k}")

    # Create datasets and loaders
    train_dataset = TensorDataset(X_cat_train, X_cont_train, y_train)
    val_dataset = TensorDataset(X_cat_val, X_cont_val, y_val)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model, loss, optimizer, and scheduler
    model = TabularModel(emb_sizes, n_cont, out_size, layers, p).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)

    # Training loop
    for epoch in range(epochs):
        model.train()
        train_losses = []

        for X_cat_batch, X_cont_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_cat_batch, X_cont_batch)
            loss = torch.sqrt(criterion(y_pred, y_batch))
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        model.eval()
        val_losses = []
        with torch.no_grad():
            for X_cat_batch, X_cont_batch, y_batch in val_loader:
                y_pred = model(X_cat_batch, X_cont_batch)
                loss = torch.sqrt(criterion(y_pred, y_batch))
                val_losses.append(loss.item())

        mean_val_loss = np.mean(val_losses)
        scheduler.step(mean_val_loss)

        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {np.mean(train_losses):.4f}, Val Loss: {mean_val_loss:.4f}")

        # Check for best model
        if mean_val_loss < best_loss:
            best_loss = mean_val_loss
            best_lr = optimizer.param_groups[0]['lr']
            best_model_state = copy.deepcopy(model.state_dict())

print(f"Best Learning Rate: {best_lr}")

Fold 1/12
Epoch 1/100, Train Loss: 9.5076, Val Loss: 8.2350
Epoch 2/100, Train Loss: 6.1683, Val Loss: 3.3245
Epoch 3/100, Train Loss: 3.1408, Val Loss: 2.5852
Epoch 4/100, Train Loss: 3.0029, Val Loss: 2.5468
Epoch 5/100, Train Loss: 2.9456, Val Loss: 2.5598
Epoch 6/100, Train Loss: 2.8658, Val Loss: 2.5057
Epoch 7/100, Train Loss: 2.8160, Val Loss: 2.5120
Epoch 8/100, Train Loss: 2.8126, Val Loss: 2.4971
Epoch 9/100, Train Loss: 2.7759, Val Loss: 2.4944
Epoch 10/100, Train Loss: 2.7595, Val Loss: 2.4792
Epoch 11/100, Train Loss: 2.7406, Val Loss: 2.4781
Epoch 12/100, Train Loss: 2.7246, Val Loss: 2.4734
Epoch 13/100, Train Loss: 2.7339, Val Loss: 2.4679
Epoch 14/100, Train Loss: 2.6977, Val Loss: 2.4689
Epoch 15/100, Train Loss: 2.6869, Val Loss: 2.4613
Epoch 16/100, Train Loss: 2.6692, Val Loss: 2.4599
Epoch 17/100, Train Loss: 2.6694, Val Loss: 2.4553
Epoch 18/100, Train Loss: 2.6657, Val Loss: 2.4588
Epoch 19/100, Train Loss: 2.6567, Val Loss: 2.4518
Epoch 20/100, Train Loss: 2.65

KeyboardInterrupt: 

In [10]:
# Function to reset weights and biases
def reset_weights(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()

# Apply the reset function to each layer
model.apply(reset_weights)

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(24, 12)
    (1): Embedding(2, 1)
    (2): Embedding(7, 4)
  )
  (emb_drop): Dropout(p=0.2, inplace=False)
  (batch_norm_cont): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=22, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=100, out_features=1, bias=True)
  )
)

### Learning schedule changing the network architecture, the drop-off probability and applying adaptively learning rate

In [11]:
# emb_sizes
# cont
out_size = 1

# Experiment with different architectures and dropout rates
arch_configs = [
    ([50, 25], 0.1),
    ([50, 25], 0.2),
    ([50, 25], 0.3),
    ([50, 25], 0.4),
    ([100, 50], 0.1),
    ([100, 50], 0.2),
    ([100, 50], 0.3),
    ([100, 50], 0.4),
    ([200, 100], 0.1),
    ([200, 100], 0.2),
    ([200, 100], 0.3),
    ([200, 100], 0.4),
    ([200, 100, 50], 0.1),
    ([200, 100, 50], 0.2),
    ([200, 100, 50], 0.3),
    ([200, 100, 50], 0.4),
]

arch_configs_r = arch_configs[::-1] # <- inverse the architecture configurations list 

batch_size = 64
epochs = 30

# Initialize a list to store results
results = []
k = 12

In [12]:
for arch, p in arch_configs_r:
    fold = 0
    best_lr = None
    best_loss = float('inf')
    
    print(f"Testing architecture: {arch} with dropout: {p}")

    for X_cat_train, X_cat_val, X_cont_train, X_cont_val, y_train, y_val in kfold_split(k, cats, conts, y):
        fold += 1
        print(f"Fold {fold}/{k}")

        # Create datasets and loaders
        train_dataset = TensorDataset(X_cat_train, X_cont_train, y_train)
        val_dataset = TensorDataset(X_cat_val, X_cont_val, y_val)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 2)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers = 2)

        # Initialize model, loss, optimizer, and scheduler
        model = TabularModel(emb_sizes, n_cont, out_size, arch, p).to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.01)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)

        # Training loop
        for epoch in range(epochs):
            model.train()
            train_losses = []

            for X_cat_batch, X_cont_batch, y_batch in train_loader:
                optimizer.zero_grad()
                y_pred = model(X_cat_batch, X_cont_batch)
                loss = torch.sqrt(criterion(y_pred, y_batch))
                loss.backward()
                optimizer.step()
                train_losses.append(loss.item())

            model.eval()
            val_losses = []
            with torch.no_grad():
                for X_cat_batch, X_cont_batch, y_batch in val_loader:
                    y_pred = model(X_cat_batch, X_cont_batch)
                    loss = torch.sqrt(criterion(y_pred, y_batch))
                    val_losses.append(loss.item())

            mean_val_loss = np.mean(val_losses)
            scheduler.step(mean_val_loss)

            print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {np.mean(train_losses):.4f}, Val Loss: {mean_val_loss:.4f}")

            # Check for best model
            if mean_val_loss < best_loss:
                best_loss = mean_val_loss
                best_lr = optimizer.param_groups[0]['lr']
                best_model_state = copy.deepcopy(model.state_dict())

        # Save results for the current fold and configuration
        results.append({
            "Architecture": str(arch),
            "Dropout": p,
            "Fold": fold,
            "Best Learning Rate": best_lr,
            "Best Validation Loss": best_loss
        })

Testing architecture: [200, 100, 50] with dropout: 0.4
Fold 1/12
Epoch 1/30, Train Loss: 3.3984, Val Loss: 2.5962
Epoch 2/30, Train Loss: 3.0996, Val Loss: 2.5279
Epoch 3/30, Train Loss: 3.0242, Val Loss: 2.4725
Epoch 4/30, Train Loss: 2.9639, Val Loss: 2.6169
Epoch 5/30, Train Loss: 2.9658, Val Loss: 2.4722
Epoch 6/30, Train Loss: 2.9258, Val Loss: 2.7005
Epoch 7/30, Train Loss: 2.9372, Val Loss: 2.4383
Epoch 8/30, Train Loss: 2.9254, Val Loss: 2.3847
Epoch 9/30, Train Loss: 2.9132, Val Loss: 2.4316
Epoch 10/30, Train Loss: 2.8808, Val Loss: 2.5447
Epoch 11/30, Train Loss: 2.8919, Val Loss: 2.4488
Epoch 12/30, Train Loss: 2.8792, Val Loss: 2.4315
Epoch 13/30, Train Loss: 2.8760, Val Loss: 2.3935
Epoch 14/30, Train Loss: 2.8937, Val Loss: 2.4107
Epoch 15/30, Train Loss: 2.8730, Val Loss: 2.5446
Epoch 16/30, Train Loss: 2.8689, Val Loss: 2.3736
Epoch 17/30, Train Loss: 2.8637, Val Loss: 2.5704
Epoch 18/30, Train Loss: 2.8487, Val Loss: 2.3565
Epoch 19/30, Train Loss: 2.8775, Val Loss: 2

KeyboardInterrupt: 